In [4917]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import OrderedDict

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4918]:
from csv_data import get_vocab_dict
from csv_data import get_sports
from csv_data import csv_to_list
from csv_data import split_data
from csv_data import extract_data

# # TODO : fjern ord der er kategorisert som "egennavn" i ddo_fullforms_2020-08-26.csv

ordered_dict = get_vocab_dict()
df_sport = get_sports()
nationalities = csv_to_list('nat3.csv')
danske_navne = csv_to_list('danmark_navne.csv')
danske_fornavne = csv_to_list('danske_fornavne.csv')
countries = csv_to_list('countries.csv')
navne = csv_to_list('navne.csv')

def get_results_in_data(train_data, train_labels):
    results = []
    for t in range(len(train_data)):
        if train_labels[t] == 1:
            results.append(train_data[t])
    
    return results


train_data, val_data, train_labels, val_labels = split_data(extract_data(df_sport), 3)

# train_data = list(train_data)
# train_labels = list(train_labels)

# for i in range(2000):
#     if train_labels[i] == 0:
#        del train_labels[i]
#        del train_data[i]

# train_data  = np.array(train_data)
# train_labels  = np.array(train_labels)


train_data_results = get_results_in_data(train_data, train_labels)

# print("Total data: ", len(train_text))
print("Train data length: ", len(train_data), len(train_labels))
print("Validation data length: ", len(val_data),  len(val_labels))

# train_data_results
# TODO : create train, validation and test set here


Train data length:  6797 6797
Validation data length:  210 210


In [4919]:
import re
train_results =  get_results_in_data(train_data, train_labels)
train_results_frequent_words = []
for tt in train_results:
    if not re.search(r'\bvinde|vinder|vundet|vandt|sejr|sejren|sejrer|tabe|tabte|taber|slår|slog|nederlag|succes|triumf|sikrer sig|sikrede sig|point|booker billet|ryger ud|ydmyger|kvalificerer|klar til|medaljer|videre|sikrer|sikret', tt, re.IGNORECASE):
        train_results_frequent_words.append(tt)

print(len(train_results))
print(len(train_results_frequent_words))
# train_results_frequent_words

2878
735


In [4920]:

# new_train = []
# new_labels = []
# for t in range(len(train_data)):
#     if "De frafaldne: Hun vandt bronze i Rio" in train_data[t]:

#         print(train_data[t])
#     else:
#         new_train.append(train_data[t])
#         new_labels.append(train_labels[t])

# train_data = np.array(new_train)
# train_labels = np.array(new_labels)

# print(len(train_data))





In [4921]:

def combine_articles_to_csv():
    df_sport_combined = df_sport.copy().drop('Link', axis=1)
    df_sport_combined.to_csv('articles_temp/combined.csv')

combine_articles_to_csv()


In [4922]:
duplicate_rows = df_sport.duplicated()
print("Duplicates in data points: ")
print(df_sport[duplicate_rows])

Duplicates in data points: 
Empty DataFrame
Columns: [Category, Headline, SubHeading, Link, isResult, isMaybe]
Index: []


In [4923]:
import time
isin_dict = False
def test_lookup_performance():
    word_to_check = "Dansk"
    start_time = time.time()

    for x in range(1000000):
        isin_dict = word_to_check in ordered_dict

    end_time = time.time()  
    assert(end_time - start_time < 1)
    print(isin_dict)

test_lookup_performance()

# isin_dict


False


In [4924]:


# train_text = df_sport.iloc[:, [0,1,2]].apply(' . '.join, axis=1).replace('\xa0', '', regex=True).to_numpy()
# train_text_results = df_sport.loc[df_sport['isResult'] == True].iloc[:, [0,1,2]].apply(' . '.join, axis=1).replace('\xa0', '', regex=True).to_numpy()

from create_vocab import split_sentences, strip_sentences
from create_vocab import remove_duplicates
from create_vocab import remove_nationalities
from create_vocab import remove_danske_navne
from create_vocab import remove_danske_fornavne
from create_vocab import remove_numeric
from create_vocab import remove_non_dict_words
from create_vocab import add_non_alpha_numeric

print(len(train_data))

# words_total =  train_data.copy()
# words_total.extend(val_data)

stripped_sentecess  = strip_sentences(train_data)
split_stences = split_sentences(stripped_sentecess)
words_arr_unique = remove_duplicates(split_stences)
words_arr_unique = remove_nationalities(words_arr_unique, nationalities)
words_arr_unique = remove_danske_navne(words_arr_unique, danske_navne)
words_arr_unique = remove_danske_fornavne(words_arr_unique, danske_fornavne)
words_arr_unique = remove_numeric(words_arr_unique)

words_train_vocab, words_sport_lingo = remove_non_dict_words(words_arr_unique, ordered_dict)



# # TODO : brug tensorflow Tokenezier til at omdanne ord til tokens
# # TODO : søg i alle leksikoner, søg med og uden bindestreg
# # TODO : håndter tal ikke i ordbøger eks ( x-x eller x-årig)
# # TODO : lemmatizer : udelad bøjninger af samme navneord. eks : verdensmester/verdensmesteren
# # TODO : evt. grupper ord der ofte hænger sammen med nltk BigramFinder. eks vandt over
# TODO : fjern evt. også alle navne (fornavne og efternavne)  

print("total unique words:", len(words_arr_unique) )
print("total sports lingo words:", len(words_sport_lingo) )
print("total vocab:", len(words_train_vocab))
print("total articles:", len(df_sport) )

# for d in df_sport['isResult']:
#     if isinstance(d, bool) != True:
#         print(d)

import collections
duplicates = [item for item, count in collections.Counter(stripped_sentecess).items() if count > 1]
print(duplicates)
assert(len(duplicates) == 0)

6797


total unique words: 15849
total sports lingo words: 3746
total vocab: 12103
total articles: 7007
[]


In [4925]:
file = open('words_sport_lingo.txt','w')
for item in words_sport_lingo:
	file.write(item+"\n")
file.close()

file = open('words_train_vocab.txt','w')
for item in sorted(words_train_vocab):
	file.write(item+"\n")
file.close()


In [4926]:

def most_frequent_words_in_arr(arr):
    xx = np.array(arr, dtype=object)
    unique, counts = np.unique(xx, return_counts=True)
    aa = np.asarray((unique, counts)).T
    return np.flip(aa[aa[:, 1].argsort()])

frequent_words = most_frequent_words_in_arr(split_stences)

# TODO : lav en negativ liste også
# display most frequent words found in lingo words
for f in frequent_words:
    if f[1] in words_sport_lingo and f[1] not in navne and len(f[1]) > 1:
        print(f[1])

# frequent_words[:100]


høgh
kesslers
mock
lolenga
volvo
america
anette
jørgen
klitschko
night
boxing
enock
vestas
froch
tysons
mocks
iba
ward
hour
holyfield
wladimir
bt
logan
sun
sejlduo
warrior
spinks
wallin
klitmøller
mathiasen
marvin
marvelous
kubrat
calle
ricky
ricard
hoya
hung
kai
pretty
franck
alis
scallywag
otto
joseph
petitjean
pulev
saul
duran
vuitton
eddie
eduard
staples
steinfath
vindhul
sorø
towell
fleischer
fisher
freddy
erfx
eubank
enerolisa
ferdie
findley
zulina
avila
zolani
take
ashley
yulihan
batman
beibi
benn
berrocal
belfast
ypremyan
bashir
barsøe
thyboes
thurø
abduqaxorov
terteryans
angulo
terence
alicia
crawford
columbianeren
wayweather
dadashev
casino
cat
calzaghe
canelo
clearwater
stjernebokser
days
bettina
buhl
brisbane
manyuch
lynggaard
luna
lvarez
maxim
mayweathers
roy
lowe
ljungquist
roamer
robeisy
poulsens
oregon
oduduru
off
nørrelund
olivia
pacheco
qudratillo
mwandila
ramirez
rafn
nobody
nights
nigel
møllenberg
nadims
heinzes
heinze
haye
hearns
hearn
skovflåtbid
gutknecht
skoglun

In [4927]:


# f = frequent_words[-300:]
ff = []
for w in frequent_words:
    if w[1] in words_train_vocab and w[0] < 2:
        ff.append(w[1])

print(len(ff))
# ff
    

5889


In [4928]:


# df_sport_labels = df_sport['isResult']

# results_true = df_sport_labels.loc[df_sport_labels== True]
# results_false = df_sport_labels.loc[df_sport_labels == False]

# assert(len(results_true) + len(results_false) == len(df_sport_labels))

# labels = df_sport_labels.to_numpy().astype(int)

# print("Labels True: " , len(results_true))
# print("Labels False: ", len(results_false))

count_positive = 0
count_negative = 0
for t in train_labels:
    if t == 0:
        count_negative += 1
    else:
        count_positive += 1

print(count_negative)
print(count_positive)




3919
2878


In [4929]:
# assert (len(labels) == len(train_text))
# print("Data:")
# for t in range(len(train_text)):
#     if labels[t] == 0:
#         print(train_text[t])
#         print(labels[t], "\n")

In [4930]:


from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from vectorization import replace_finals, replace_nationality, to_lower
from vectorization import split_dash
from vectorization import split_included_specials
from vectorization import replace_tournament
from vectorization import replace_countries
from vectorization import replace_weekday
from vectorization import replace_digits
from vectorization import vect_layer_2_text
from vectorization import vectorize_layer
from vectorization import standardize
from static_data import tournaments
from static_data import weekdays
from static_data import non_alpha
from static_data import word_generalization


# TODO : evt indikere hvilke navneord der starte med stort bogstav(egenavne), evt. lave et opslag for at undersøge ordklasse for det første ord i sætningen 
# TODO : test hvilke standarization funktioner giver bedre resultater 

arrs = [
    to_lower, 
    split_dash, 
    split_included_specials, 
    replace_tournament(tournaments),
    replace_countries(countries), 
    replace_weekday(weekdays), 
    replace_finals,
    replace_nationality(nationalities),
    replace_digits
]

s = standardize(arrs)

words_train_vocab.extend(word_generalization)
words_train_vocab.extend(non_alpha)

# Model constants.
max_features = 12200
sequence_length = 70

vectorized_layer = vectorize_layer(max_features, sequence_length, s)
print(words_train_vocab)
text_ds = vectorized_layer.adapt(words_train_vocab)
vect_vocab = vectorized_layer.get_vocabulary()

print("Total vocab/max_features : ",  len(vect_vocab))
# print (vect_layer_2_text(vectorized_layer(["OL Paris 2023 Fodbold superliga pokalen maraton-majors maraton-maj åmaj majå torsdags torsdage superligaer pokalfinaler semifinaler finalen finalerne"]), vect_vocab))

# print (vect_layer_2_text(vectorized_layer(["majoren bla major league bla major"]), vect_vocab))
# print (vect_layer_2_text(vectorized_layer(["pga PGA pga-turneringerne pga turneringens PGA-turneringen pga-turneringer turnering turneringer"]), vect_vocab))

# print (vect_layer_2_text(vectorized_layer(["vuelta a españa bla 1933 _i .den 10."]), vect_vocab))

# is_match = tf.strings.regex_replace(
#     "1820. Tomorrow is 1938 another day 1938", pattern=r'(?:18|19|20)\d{2}', rewrite=r'x_year', replace_global=True
# )

# is_match

# TODO : Test for all non alpha numeric signs , eg », «

# print(weekdays)


# print (vect_layer_2_text(vectorized_layer(["én to tre Den allerførste anden  andende danske verdensetter skuffede i VM-kvartfinalen håndboldherrerne fodboldligaen fodboldspillere i trines land er trine Italiensk tempotog træder fra både Vingegaard og verdensmesteren på Vuelta-enkeltstart"]), vect_vocab))
# 

print (vect_layer_2_text(vectorized_layer(["'Golf . Dansker laver historisk hole-in-one i stor turnering . Europa har vundet turneringen de seneste to år"]), vect_vocab))



['kampsport', 'skræddersyet', 'udgangspunkter', 'tabe', 'underordnet', 'målmandssituationen', 'håndbolddamerne', 'drøm', 'overmagt', 'femdobbelte', 'næseblod', 'måneden', 'langsomst', 'betaler', 'parrets', 'veteraner', 'junge', 'stole', 'efterlyste', 'klip', 'europamestertitel', 'spoleret', 'fremprovokerede', 'etapevinder', 'motorproblemer', 'fristil', 'hober', 'dingle', 'sherif', 'sejrstørke', 'bugt', 'skidt', 'kane', 'nedsat', 'højdespringeren', 'manifesterede', 'kvalifikationsturneringer', 'linden', 'bølgen', 'pacemaker', 'majoren', 'topligaer', 'erstatte', 'skæmmer', 'dagens', 'kulmination', 'livet', 'sidstedag', 'fireårige', 'pointe', 'scoringen', 'positivitet', 'etapen', 'liren', 'røgen', 'sensationsvinder', 'mestre', 'top', 'præ', 'torsdag', 'trofæ', 'forstærker', 'ørken', 'motorsportsverdenen', 'størst', 'toptennis', 'landets', 'kontraktens', 'forfremmer', 'løbetræner', 'opsamlingsheat', 'førstnævntes', 'atiletikforbund', 'skoleskyderi', 'svigende', 'pillede', 'isolerede', 'cor

In [4931]:
# for t in train_data[0:50]:
#     print("Original \n:", t)
#     print("Text from vectorized: \n", vect_layer_2_text(
#         vectorized_layer([t]), vect_vocab
#         ))
#     print("\n")


In [4932]:
# def split_data(data, labels, percentage):
#     l = len(data)
#     p = l - int((percentage/100) * l)
#     return (data[0:p], data[p:], labels[0:p], labels[p:])


# train_data, val_data, train_labels, val_labels = split_data(vectorize_layer(train_text), labels, 7)

# print("Total data: ", len(train_text))
# print("Train data length: ", len(train_data))
# print("Validation data length: ", len(val_data))


In [4933]:
train_data_vect = vectorized_layer(train_data)
val_data_vect = vectorized_layer(val_data)



In [4934]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.4):
        super().__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions



In [4935]:
from tensorflow.keras import layers
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [4936]:
from tensorflow.keras import layers
import random as python_random


def get_transformer_model():

    embed_dim =  58 # Embedding size for each token
    num_heads = 2  # Number of attention heads
    ff_dim = 24  # Hidden layer size in feed forward network inside transformer


    # A integer input for vocab indices.
    inputs = tf.keras.Input(shape=(sequence_length,), dtype="int64")

    # Next, we add a layer to map those vocab indices into a space of dimensionality
    # 'embedding_dim'.
    # x = layers.Embedding(max_features, embed_dim)(inputs)

    embedding_layer = TokenAndPositionEmbedding(sequence_length, max_features, embed_dim)
    x = embedding_layer(inputs)
    x = layers.Dropout(0.5)(x)
    # x = layers.Dropout(0.2)(x)

    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    # x = layers.Dropout(0.1)(x)

    # x = transformer_block(x)



    # Conv1D + global max pooling
    # x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)
    # x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)


    x = layers.GlobalMaxPooling1D()(x)
    # x = layers.Dropout(0.5)(x)

    # We add a vanilla hidden layer:
    # x = layers.Dense(32, activation="relu")(x)
    # x = layers.Dropout(0.5)(x)

    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)


    transformer_model = tf.keras.Model(inputs, predictions)

    # Compile the model with binary crossentropy loss and an adam optimizer.
    transformer_model.compile(loss="binary_crossentropy", optimizer= "adamax", metrics=["accuracy"])
    return transformer_model


In [4937]:
from tensorflow.keras import layers
import random as python_random

def get_cnn_model():

    embedding_dim = 150

    # A integer input for vocab indices.
    inputs = tf.keras.Input(shape=(None,), dtype="int64")

    # Next, we add a layer to map those vocab indices into a space of dimensionality
    # 'embedding_dim'.
    x = layers.Embedding(max_features, embedding_dim)(inputs)
    # x = layers.Dropout(0.5)(x)

    # Conv1D + global max pooling
    # x = layers.Conv1D(32, 11, padding="valid", activation="relu")(x)
    # x = layers.Conv1D(128, 9, padding="valid", activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    # Conv1D + global max pooling
    x = layers.Conv1D(64, 7, padding="valid", activation="relu", strides=3)(x)
    x = layers.Conv1D(64, 7, padding="valid", activation="relu", strides=3)(x)





    x = layers.GlobalMaxPooling1D()(x)

    # We add a vanilla hidden layer:
    # x = layers.Dense(128, activation="relu")(x)
    # x = layers.Dropout(0.5)(x)
    # We add a vanilla hidden layer:
    # x = layers.Dense(128, activation="relu")(x)
    # x = layers.Dropout(0.5)(x)
    # x = layers.Dropout(0.5)(x)

    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)
    
    cnn_model = tf.keras.Model(inputs, predictions)

    # Compile the model with binary crossentropy loss and an adam optimizer.
    cnn_model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), metrics=["accuracy"])
    return cnn_model



In [4938]:

def prepare_model(name):
    if (name == "cnn"):
       return get_cnn_model()
    elif (name == "transformer"):
       return get_transformer_model()
  

def filter_max_accuracy(history, threshold = 0.95):
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    list = []
    for x in range(len(acc)):
        if (acc[x] > threshold):
            list.append(val_acc[x])

    return np.array(list)

models = ["cnn", "transformer"]

callback_3_loss = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4)


def mean_model_accuracy(mode_names, iterations, epochs = 20):

  
    results = []

    for name in range(len(mode_names)):
        model_name = mode_names[name]
        val_accuracies = []
        
        for x in range(iterations):
            model = prepare_model(model_name)

            # Fit the model using the train and test datasets.
            history = model.fit(train_data_vect, train_labels, epochs=epochs, batch_size=6, validation_data=(val_data_vect, val_labels), callbacks=[callback_3_loss])

            max_val_acc = filter_max_accuracy(history)
            val_accuracies.append(max(max_val_acc))
            print(max(max_val_acc))
            print(val_accuracies)
        
        d = dict(name = model_name, results = np.mean(np.squeeze(np.array(val_accuracies))))
        results.append(d)
        
    return results


In [4939]:
# mean_results = mean_model_accuracy(models, 8)
# mean_results

In [4940]:
def result_format_round(result):
    return round(result)

def result_format_none(result):
    return result

def print_model_score(model):
    score = model.evaluate(val_data_vect, val_labels, verbose=0)
    print("Validation loss:", score[0])
    print("Validations accuracy:", score[1])

def print_validation_results(predictions, val_data, labels, formatter, only_incorrect = False):
    print("Number of predictions", len(predictions))
    n_correct = 0
    for x in range(len(val_data)):
        correct_prediction = result_format_round(labels[x]) == result_format_round(predictions[x][0])
        if correct_prediction:
            n_correct += 1

        if correct_prediction == False and result_format_round(labels[x]) == 0:
            print("VALIDATION SAMPLE TEXT: \n" ,val_data[x])
            print("VALIDATION SAMPLE DE-VECTORIZED: \n" ,vect_layer_2_text(val_data_vect[x], vect_vocab))
            print("LABEL --:" , formatter(labels[x]), " ---- float: ", labels[x])
            print("PREDICTION --:" , formatter(predictions[x][0]), " ---- float: ", predictions[x][0])
            print("CORRECT PREDICTION: ", correct_prediction)
            print("\n")
    
    print("Number correct: ", n_correct)

In [4941]:

epochs= 30
transformer_model = get_transformer_model()


# Fit the model using the train and test datasets.
transformer_history = transformer_model.fit(train_data_vect, train_labels, epochs=epochs, batch_size=40, validation_data=(val_data_vect, val_labels),callbacks=[callback_3_loss])


Epoch 1/30
170/170 [==============================] - 2s 9ms/step - loss: 0.7111 - accuracy: 0.5701 - val_loss: 0.7823 - val_accuracy: 0.5810
Epoch 2/30
170/170 [==============================] - 1s 8ms/step - loss: 0.6125 - accuracy: 0.6703 - val_loss: 0.5653 - val_accuracy: 0.7000
Epoch 3/30
170/170 [==============================] - 1s 8ms/step - loss: 0.5493 - accuracy: 0.7253 - val_loss: 0.4534 - val_accuracy: 0.7952
Epoch 4/30
170/170 [==============================] - 1s 8ms/step - loss: 0.4873 - accuracy: 0.7755 - val_loss: 0.3854 - val_accuracy: 0.8571
Epoch 5/30
170/170 [==============================] - 1s 8ms/step - loss: 0.4519 - accuracy: 0.7917 - val_loss: 0.3469 - val_accuracy: 0.8714
Epoch 6/30
170/170 [==============================] - 1s 8ms/step - loss: 0.4081 - accuracy: 0.8186 - val_loss: 0.3052 - val_accuracy: 0.8905
Epoch 7/30
170/170 [==============================] - 1s 8ms/step - loss: 0.3786 - accuracy: 0.8382 - val_loss: 0.2784 - val_accuracy: 0.9000
Epoch 

In [4942]:

# epochs= 8
# cnn_model = get_cnn_model()


# # # Fit the model using the train and test datasets.
# transformer_history = cnn_model.fit(train_data_vect, train_labels, epochs=epochs, batch_size=132, validation_data=(val_data_vect, val_labels), callbacks=[callback_3_loss])

In [4943]:
def print_results(model):
    np.set_printoptions(precision = 5, suppress = True)
    predictions = model.predict(val_data_vect)
    print_model_score(model)
    print("\n")
    print_validation_results(predictions, val_data, val_labels, result_format_round)
  

In [4944]:
print("--- TRANSFORMER ---")
print_results(transformer_model)

--- TRANSFORMER ---


Validation loss: 0.21369585394859314
Validations accuracy: 0.9285714030265808


Number of predictions 210
VALIDATION SAMPLE TEXT: 
 Basketball . Uheldig NBA-stjerne fik slået tand i stykker . Jaylen Brown var uheldig i kampen mod Milwaukee Bucks, hvor han fik slået en tand i stykker
VALIDATION SAMPLE DE-VECTORIZED: 
 ['basketball' '.' 'uheldig' 'xtournament' 'stjerne' 'fik' 'slået' 'tand'
 'i' 'stykker' '.' '[UNK]' '[UNK]' 'var' 'uheldig' 'i' 'kampen' 'mod'
 '[UNK]' '[UNK]' 'hvor' 'han' 'fik' 'slået' 'en' 'tand' 'i' 'stykker']
LABEL --: 0  ---- float:  0
PREDICTION --: 1  ---- float:  0.85124195
CORRECT PREDICTION:  False


VALIDATION SAMPLE TEXT: 
 CHAMPIONS LEAGUE . Han har vundet det hele, men nu drømmer Landin om det største med Aalborg . Niklas Landin og Aalborg Håndbold tager onsdag hul på Champions League 
VALIDATION SAMPLE DE-VECTORIZED: 
 ['xtournament' '.' 'han' 'har' 'vundet' 'det' 'hele' 'men' 'nu' 'drømmer'
 '[UNK]' 'om' 'det' 'største' 'med' '[UNK]' '.' '[UNK]' '[UNK]' 'o

In [4945]:

# print("--- CNN ---")
# print_results(cnn_model)

In [4946]:
import os

# Set up a logs directory, so Tensorboard knows where to look for files.

ll = transformer_model.layers[1]
ll_weights = ll.get_weights()[0]

# print(ll_weights.shape)
ll_weights


array([[-0.1271 , -0.02146,  0.02422, ..., -0.01132,  0.01096, -0.07059],
       [-0.14067, -0.03814, -0.00406, ..., -0.03032,  0.05252, -0.11727],
       [ 0.06737,  0.06346,  0.01934, ..., -0.01757, -0.05099, -0.00678],
       ...,
       [-0.00302, -0.02204,  0.01926, ..., -0.0444 , -0.02041, -0.04497],
       [ 0.00955,  0.04419,  0.01315, ...,  0.03621,  0.0491 ,  0.01571],
       [ 0.04308,  0.00653, -0.02802, ...,  0.02457, -0.01944, -0.04488]],
      dtype=float32)

In [4947]:
##import I/O module in python
import io

##open the text stream for vectors
vectors = io.open('vectors.tsv', 'w', encoding='utf-8')

##open the text stream for metadata
meta = io.open('meta.tsv', 'w', encoding='utf-8')


##write each word and its corresponding embedding
for index in range(1, len(vect_vocab)):
  word = vect_vocab[index]  # flipping the key-value in word_index
  embeddings = ll_weights[index]
  meta.write(word + "\n")
  vectors.write('\t'.join([str(x) for x in embeddings]) + "\n")

##close the stream
vectors.close()
meta.close()

In [4948]:
# from nltk import collocations
# bigram_measures = collocations.BigramAssocMeasures()
# finder = collocations.BigramCollocationFinder.from_words(["New", "York", "is", "big", "New", "York", "is", "dirty"])
# finder.ngram_fd.items()



In [4949]:
# import lemmy
# # Create an instance of the standalone lemmatizer.
# lemmatizer = lemmy.load("da")

# # Find lemma for the word 'akvariernes'. First argument is an empty POS tag.
# lemmatizer.lemmatize("NOUN", "verdensetter")



In [4950]:
# import nltk as nltk
# # from string import punctuation
# # from nltk.corpus import stopwords
# # nltk.download('stopwords')

# # da_stopwords = stopwords.words("danish")


In [4951]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorized_layer(inputs)
# Turn vocab indices into predictions
outputs = transformer_model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="transformer_crossentropy", optimizer="adam", metrics=["accuracy"]
)


In [4952]:
print("\nResults:")


print(end_to_end_model.predict(
    [
      "Fodbold . Fjerritslev vinder over Vordingborg. Træner kommenterer på historisk kamp",
       "SPORT . Hun vandt bronze i mandags Roer Anne Dsane Andersen har som 24-årig vundet bronze ved OL",
       "Badminton . Axelsen frustreret over nederlag. Viktor Axelsen trænger til ferie efter nedturen",
      "OL . Det blev til en flot medalje til Malene dfhsds. 'Jeg er meget lykkelig for resultatet'",
      "Badminton . To danskere har kvalificeret sig til finalerne. Øverst i artiklen kan du også se en video fra kampene",
     ]))


print("\n NON-Results:") 
print(end_to_end_model.predict(
    [
      "OL Meget skal ske før en medalje kommer inden for rækkevidde. Dressurrytter Malene dsds har mistet troen på success",
      "Fodbold . Træner for Fjerritslev ser frem til sejr over Vordingborg. 'Det bliver en historisk kamp'",
      "Fodbold . De danske spillere skal op imod Sverige, som de tabte til i 2022",
      "Fodbold . De danske spillere forsøge at besejre Tyrkiet den kommende Lørdag i VM-kamp. Tyrkiet har aldriv været i en VM-finale",
      "Fodbold . De danske spillere tror på sejr mod Tyrkiet. 'Den skal vindes'",
      "Skisport . Sverige drømmer om flere medaljer og sejre til næste års OL . Træner forventer flere gode resultater",
      "Boksning . Kesler vil overraske alle og gøre det umulige. 'Jeg vinder i VM'",
      "Boksning . Kesler med stor selvtillid: 'Det bliver guld eller sølv til VM'",
      "FODBOLD . Superliga-profil kan se frem til en længere pause . Den danske stærke anfører Andy Robertson skal opereres og er ukampdygtig i flere måneder",
      "EM HÅNDBOLD . Hvornår var det nu, det var? De danske håndboldkvinder husker ikke Danmarks seneste finale . Sidst det danske håndboldlandshold for kvinder spillede finale var i 2004",
       "Fodbold . Robert dfd er enormt stolt . Kasper Hjulmand satte sin lid til Robert dfdf, der scorede to gange"
     ]))


Results:
[[0.96339]
 [0.99985]
 [0.99999]
 [0.9964 ]
 [0.00366]]

 NON-Results:
[[0.00823]
 [0.16588]
 [0.30317]
 [0.01854]
 [0.02653]
 [0.00119]
 [0.6675 ]
 [0.97715]
 [0.00009]
 [0.36983]
 [0.37177]]


In [4953]:
print("\n IN-BETWEEN-Results:") 
print(end_to_end_model.predict(
    [
       "Fodbold . Fjerritslev vandt i lørdags over Vordingborg 1-0. Den danske anfører dasdad dasdasd triumferer",
       "Fodbold . Fjerritslev vandt i lørdags over Vordingborg. Den danske anfører adasdasdd daddas triumferer",
       "Fodbold . Fjerritslev vandt i lørdags over Vordingborg. Den danske anfører sdfd sdfdf har meddelt, at han skal på hospitalet",
       "Fodbold . Superliga-profil vandt i lørdags over Vordingborg. Den danske anfører fdfd fddsf skal opereres og er ude i flere måneder",
       "Fodbold . Superliga-profil har meddelelse efter sejr over Vordingborg. ' Den danske anfører fsdsdff sdffsd skal opereres og er ude i flere måneder",
       "Fodbold . Superliga-profil har meddelelse efter sejr. Den danske anfører fdfd sdffdf skal opereres og er ude i flere måneder",
       "Fodbold . Superliga-profil kan se frem til en længere pause. Den danske anfører fdfd sdfff skal opereres og er ude i flere måneder",
     ]))


 IN-BETWEEN-Results:
[[0.99998]
 [0.99996]
 [0.57205]
 [0.88366]
 [0.52601]
 [0.48765]
 [0.00011]]
